Transcripts: https://survivor.fandom.com/wiki/The_Marooning/Transcript  
Scores: https://www.truedorktimes.com/survivor/boxscores/s5.htm  

In [1]:
import pandas as pd
import numpy as np
import re

import requests
from bs4 import BeautifulSoup

import openpyxl
from io import BytesIO

## Loop for All Seasons / Episodes

In [2]:
seasons = range(1,44)

In [3]:
df_dict_raw = {}

for szn in seasons:
    main_url = 'https://www.truedorktimes.com/survivor/boxscores/s'+str(szn)+'.htm'
    page = requests.get(main_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    doc_id = re.search(r'(?<=https://docs.google.com/spreadsheets/d/).*(?=(/edit?|/">))',
                   str(soup)).group()
    
    doc_link = f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx'
    page = requests.get(doc_link)
    excel_data = BytesIO(page.content)
    excel_file = openpyxl.load_workbook(excel_data)
    
    episodes = [x for x in excel_file.sheetnames if 'season' not in x if 'Glossary' not in x]
    
    for epi in episodes:
    
    ###
        if szn == 31 and epi == 'ep1':
            continue
    ###
        
        df_name = 's'+str(szn)+epi
        df_dict_raw[df_name] = pd.read_excel(excel_data, sheet_name = epi)
        
    if szn % 5 == 0:
        print('Season',szn,'complete.')

Season 5 complete.
Season 10 complete.
Season 15 complete.
Season 20 complete.
Season 25 complete.
Season 30 complete.
Season 35 complete.
Season 40 complete.


In [4]:
df_dict_clean = {}

for df_name in df_dict_raw.keys():
    
    szn = int(re.search(r'(?<=s)[0-9]+', df_name).group())
    epi = int(re.search(r'(?<=e)[0-9]+', df_name).group())
    
    ###
    if df_name == 's31e1':
        continue
    ###
    
    epi_data = df_dict_raw[df_name]
    cutoff = epi_data.index[epi_data['Unnamed: 0'].isnull()].min()

    df1 = (epi_data
           .iloc[0:cutoff, 0:16]
           .query('`Unnamed: 0`.notnull()')
           .reset_index(drop = True)
           .rename(columns = {'Unnamed: 0':'player_name',
                'ChW':'challenge_wins',
                'ChA':'challenge_appearances',
                'SO':'sit_outs',
                'VFB':'votes_for_bootee',
                'VAP':'votes_against_player',
                'TotV':'total_votes',
                'TCA':'tribal_council_appearances',
                'tot days':'total_days',
                'exile days':'exile_days'
               })
            .filter(['player_name','challenge_wins','challenge_appearances','sit_outs',
                     'votes_for_bootee','votes_against_player','total_votes',
                     'tribal_council_appearances','total_days','exile_days'], axis = 1)
          )

    df2 = (epi_data
           .iloc[cutoff:len(epi_data), 0:7]
           .query('`Unnamed: 0`.notnull()')
           .reset_index(drop = True)
           .rename(columns = {'Unnamed: 0':'player_name',
                'ChW':'reward_chl_win',
                'ChA':'reward_chl_teammates',
                'SO':'reward_chl_win_perc',
                'Unnamed: 4':'immun_chl_win',
                'VFB':'immun_chl_teammates',
                'VAP':'immun_chl_win_perc'
               })
          )

    combo = (df1
             .merge(df2, how = 'left', on = 'player_name')
             .replace(r'\*[A-Za-z]+',np.nan, regex = True)
             .dropna(thresh = 2)
             .assign(season = szn,
                     episode = epi)
            )

    df_dict_clean[df_name] = combo
    
    if szn % 10 == 0 and epi == 1:
        print('Season',szn,'complete.')

Season 10 complete.
Season 20 complete.
Season 30 complete.
Season 40 complete.


In [5]:
full_df = pd.DataFrame(columns = combo.columns)

for i in df_dict_clean:
    full_df = pd.concat([full_df,df_dict_clean[i]],
                        ignore_index = True)
    
full_df = full_df.dropna(axis = 1, how = 'all')

In [6]:
full_df

,player_name,challenge_wins,challenge_appearances,sit_outs,votes_for_bootee,votes_against_player,total_votes,tribal_council_appearances,total_days,exile_days,reward_chl_win,reward_chl_teammates,reward_chl_win_perc,immun_chl_win,immun_chl_teammates,immun_chl_win_perc,season,episode
0,Richard,0.0,1.000000,NaN,1.0,NaN,3.0,1.0,2.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1,13
1,Kelly W,1.0,1.000000,NaN,1.0,NaN,3.0,1.0,2.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1,13
2,Rudy,0.0,1.000000,NaN,0.0,1.0,3.0,1.0,1.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1,13
3,Richard,0.0,1.000000,NaN,1.0,2.0,4.0,1.0,1.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1,13
4,Kelly W,1.0,1.000000,NaN,0.0,NaN,4.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7126,Nicole Capper,0.0,0.142857,NaN,1.0,NaN,7.0,1.0,3.0,NaN,NaN,NaN,NaN,0.0,7.0,0.0,43,1
7127,Paul Smulders,0.0,0.142857,NaN,0.0,NaN,7.0,1.0,3.0,1.0,NaN,NaN,NaN,0.0,7.0,0.0,43,1
7128,Rob Bentele,0.0,0.142857,NaN,1.0,NaN,7.0,1.0,3.0,NaN,NaN,NaN,NaN,0.0,7.0,0.0,43,1
7129,Seipei Mashugane,0.0,0.142857,NaN,1.0,2.0,7.0,1.0,3.0,NaN,NaN,NaN,NaN,0.0,7.0,0.0,43,1


In [7]:
full_df.to_csv('data/survivor_boxscores.csv', index = False)